In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

# 0. Notebook description

In this notebook, we load our cleaned dataset and perform a content-based recommendation based on the `Genre` column, which contains one or more movie genres.

# 1. Load dataset

In [2]:
movies_df = pd.read_csv('datasets/imdb_top_1000_cleaned.csv', low_memory=False)

movies_df.head()

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142.0,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,28341469.0
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175.0,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,134966411.0
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152.0,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,534858444.0
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202.0,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,57300000.0
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96.0,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,4360000.0


# 2. Prepare tf-idf model for the `Genre` column

The Genre column contains the genre(s) of each movie, which we used to build a content-based recommendation system. We transformed the genres into numeric vector representations using a multi-label binarizer.

This process creates a binary matrix where each row represents a movie and each column represents a genre, with 1 indicating the movie belongs to that genre and 0 otherwise. This representation was used as input for the recommendation system.

In [10]:
# print the Genre column of the first 10 movies
print(movies_df['Genre'].head(10))

0                        Drama
1                 Crime, Drama
2         Action, Crime, Drama
3                 Crime, Drama
4                 Crime, Drama
5     Action, Adventure, Drama
6                 Crime, Drama
7    Biography, Drama, History
8    Action, Adventure, Sci-Fi
9                        Drama
Name: Genre, dtype: object


In [4]:
vectorizer = TfidfVectorizer(stop_words='english', max_df=0.8, min_df=2)

movies_df.Genre = movies_df.Genre.fillna('')
tfidf_model = vectorizer.fit_transform(movies_df.Genre)
print(f'Matrix contains {tfidf_model.shape[0]} movies and {tfidf_model.shape[1]} words')

Matrix contains 1000 movies and 23 words


### Inspect the tf-idf model

Inspect the columns with popular movie genres like 'drama', 'fantasy', 'adventure', 'comedy', 'biography', 'crime' etc.

In [5]:
columns = vectorizer.get_feature_names_out()
print(columns)

['action' 'adventure' 'animation' 'biography' 'comedy' 'crime' 'drama'
 'family' 'fantasy' 'fi' 'film' 'history' 'horror' 'music' 'musical'
 'mystery' 'noir' 'romance' 'sci' 'sport' 'thriller' 'war' 'western']


In [6]:
genres = ['drama', 'fantasy', 'adventure', 'comedy', 'biography', 'crime']
columns = vectorizer.get_feature_names_out()
tfidf_model_df = pd.DataFrame.sparse.from_spmatrix(tfidf_model, columns=columns)
tfidf_model_df[genres].head()

,drama,fantasy,adventure,comedy,biography,crime
0,1.0,0,0,0,0,0
1,0.458764,0,0,0,0,0.888558
2,0.337068,0,0,0,0,0.652851
3,0.458764,0,0,0,0,0.888558
4,0.458764,0,0,0,0,0.888558


# 3. Find similar movies

To find similar movies, we use the KNN algorithm with **cosine similarity** as a distance metric to find the nearest neighbours.

In [7]:
def get_content_based_recommendation_genre(title, top_n=10, metric='cosine'):
    # Get the index of the movie that matches the title
    # we'll use that index to locate the row in the tf-idf matrix that corresponds to that movie
    idx = movies_df[movies_df.Series_Title.str.lower() == title.lower()].index[0]

    model = NearestNeighbors(n_neighbors=top_n+1, metric=metric)
    model.fit(tfidf_model)
    similar_movies = model.kneighbors(tfidf_model[idx], return_distance=False)[0]
    similar_movies = similar_movies[1:]  # remove the first item which is the movie itself

    # Return the top 10 most similar movies
    return movies_df.iloc[similar_movies]

In [8]:
get_content_based_recommendation_genre('The Godfather')[['Series_Title', 'Genre', 'IMDB_Rating', 'No_of_Votes', 'Genre']]

,Series_Title,Genre,IMDB_Rating,No_of_Votes,Genre
480,Bir Zamanlar Anadolu'da,"Crime, Drama",7.9,41995,"Crime, Drama"
1,The Godfather,"Crime, Drama",9.2,1620367,"Crime, Drama"
22,Cidade de Deus,"Crime, Drama",8.6,699256,"Crime, Drama"
111,Taxi Driver,"Crime, Drama",8.3,724636,"Crime, Drama"
763,This Is England,"Crime, Drama",7.7,115576,"Crime, Drama"
974,The Godfather: Part III,"Crime, Drama",7.6,359809,"Crime, Drama"
108,Scarface,"Crime, Drama",8.3,740911,"Crime, Drama"
895,Leviafan,"Crime, Drama",7.6,49397,"Crime, Drama"
71,Once Upon a Time in America,"Crime, Drama",8.4,311365,"Crime, Drama"
519,Once Were Warriors,"Crime, Drama",7.9,31590,"Crime, Drama"


In [9]:
get_content_based_recommendation_genre('The Dark Knight')[['Series_Title', 'Genre', 'IMDB_Rating', 'No_of_Votes', 'Genre']]

,Series_Title,Genre,IMDB_Rating,No_of_Votes,Genre
241,Kill Bill: Vol. 1,"Action, Crime, Drama",8.1,1000639,"Action, Crime, Drama"
896,Hell or High Water,"Action, Crime, Drama",7.6,204175,"Action, Crime, Drama"
901,End of Watch,"Action, Crime, Drama",7.6,228132,"Action, Crime, Drama"
663,The Fugitive,"Action, Crime, Drama",7.8,267684,"Action, Crime, Drama"
918,Eastern Promises,"Action, Crime, Drama",7.6,227760,"Action, Crime, Drama"
674,Dip huet seung hung,"Action, Crime, Drama",7.8,45624,"Action, Crime, Drama"
605,Ajeossi,"Action, Crime, Drama",7.8,62848,"Action, Crime, Drama"
55,Vikram Vedha,"Action, Crime, Drama",8.4,28401,"Action, Crime, Drama"
577,Udta Punjab,"Action, Crime, Drama",7.8,27175,"Action, Crime, Drama"
931,Lord of War,"Action, Crime, Drama",7.6,294140,"Action, Crime, Drama"
